In [ ]:
def add_padding(dataset=None, pad_width = 0, constant_values=0) -> []:
    padded_dataset = []
    
    for i in dataset:
        padded = np.pad(i, pad_width=pad_width, mode='constant', constant_values=constant_values)
        padded_dataset.append(padded[:, :, np.newaxis])
    
    return padded_dataset

In [1]:
import pandas as pd 
import numpy as np

from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from keras.datasets import mnist
from keras.applications.vgg16 import VGG16

import matplotlib.pyplot as plt
%matplotlib inline

Loading the dataset and adding the padding

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

new_x_train = []
new_x_test = []

for x in x_train:
    x_padded = np.pad(x, pad_width=2, mode='constant', constant_values=0)
    new_x_train.append(x_padded)

for x in x_test:
    x_padded = np.pad(x, pad_width=2, mode='constant', constant_values=0)
    new_x_test.append(x_padded[:, :, np.newaxis])

In [4]:
new_x_train = np.asarray(new_x_train)
new_x_test = np.asarray(new_x_test)

dim_data = np.prod(new_x_train.shape[1:])

new_x_train = new_x_train.reshape(new_x_train.shape[0], dim_data)
new_x_train = new_x_train.astype('float32')

new_x_test = new_x_test.reshape(new_x_test.shape[0], dim_data)
new_x_test = new_x_test.astype('float32')

Creating the model

In [7]:
model = Sequential()

model.add(Dense(1024, input_shape=(dim_data,)))
model.add(Dropout(0.05))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(512, activation='linear'))

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 1024)              1049600   
                                                                 
 dropout_3 (Dropout)         (None, 1024)              0         
                                                                 
 dense_5 (Dense)             (None, 512)               524800    
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 100)               51300     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_7 (Dense)             (None, 512)              

In [25]:
model.compile(loss='mse', 
              optimizer='adam', 
              metrics=['mse'])

earlyStopping = EarlyStopping(monitor='val_loss', 
                              patience=25)

history = model.fit(new_x_train,
                    y_train,
                    epochs=250, 
                    batch_size=32, 
                    verbose=0, 
                    validation_split=0.2, 
                    callbacks=[earlyStopping])

KeyboardInterrupt: 